In [38]:
import cv2
import os

# Create a folder for the user's dataset
def create_dataset_dir(name):
    path = f"face_dataset/{name}"
    if not os.path.exists(path):
        os.makedirs(path)
    return path

# Capture face images for the given name and save them in the dataset folder
def capture_images(name):
    cap = cv2.VideoCapture(0)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    dataset_path = create_dataset_dir(name)
    img_id = 0

    print(f"Capturing images for {name}. Press ESC to stop.")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

        for (x, y, w, h) in faces:
            img_id += 1
            face = gray[y:y+h, x:x+w]
            file_name_path = os.path.join(dataset_path, f"{img_id}.jpg")
            cv2.imwrite(file_name_path, face)

            # Display the face and ID in the live camera feed
            cv2.putText(frame, str(img_id), (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        cv2.imshow("Face Capture", frame)

        if cv2.waitKey(1) == 27 or img_id >= 100:  # ESC to stop or after 100 images
            break

    cap.release()
    cv2.destroyAllWindows()

# Main function to capture images
name = input("Enter the name of the person: ")
capture_images(name)


Capturing images for Siddhartha. Press ESC to stop.


In [44]:
import cv2
import numpy as np
import os

# Function to get face images and labels for training
def get_images_and_labels(dataset_path):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    face_samples = []
    ids = []

    # Loop over each image in the person's folder
    for image_file in os.listdir(dataset_path):
        if image_file.endswith('.jpg'):
            image_path = os.path.join(dataset_path, image_file)
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            faces = face_cascade.detectMultiScale(img, scaleFactor=1.3, minNeighbors=5)

            for (x, y, w, h) in faces:
                face_samples.append(img[y:y+h, x:x+w])
                ids.append(int(os.path.split(image_file)[-1].split('.')[0]))

    return face_samples, ids

# Train the face recognition model and save it as person_name.yml
def train_model(person_name):
    dataset_path = f'face_dataset/{person_name}'
    if not os.path.exists('face_trainer'):
        os.makedirs('face_trainer')

    recognizer = cv2.face.LBPHFaceRecognizer_create()

    faces, ids = get_images_and_labels(dataset_path)
    
    if len(faces) > 0 and len(ids) > 0:
        recognizer.train(faces, np.array(ids))
        model_file_path = f'face_trainer/{person_name}.yml'
        recognizer.save(model_file_path)
        print(f"Model for {person_name} saved as {model_file_path}")
    else:
        print(f"No valid images to train on for {person_name}.")

# Main function to train the model
person_name = input("Enter the name of the person to train: ")
train_model(person_name)


Model for Siddhartha saved as trainer/Siddhartha.yml


In [55]:
import cv2
import os

# Recognize faces using the trained LBPH model
def recognize_faces(person_name):
    model_file_path = f'face_trainer/{person_name}.yml'
    
    # Check if the person's model exists
    if not os.path.exists(model_file_path):
        print(f"Model for {person_name} not found. Please train the model first.")
        return

    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read(model_file_path)

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    cap = cv2.VideoCapture(0)

    print(f"Recognizing {person_name}. Press ESC to stop.")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

        for (x, y, w, h) in faces:
            face_region = gray[y:y+h, x:x+w]
            face_id, confidence = recognizer.predict(face_region)

            if confidence < 60:  # Adjust confidence threshold as needed
                cv2.putText(frame, f"Matched (Confidence: {round(100 - confidence)}%)", (x+5, y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            else:
                cv2.putText(frame, "Not Matched", (x+5, y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        cv2.imshow("Face Recognition", frame)

        if cv2.waitKey(1) & 0xFF == 27:  # ESC to break
            break

    cap.release()
    cv2.destroyAllWindows()

# Main function to recognize faces
person_name = input("Enter the name of the person to recognize: ")
recognize_faces(person_name)


Recognizing Siddhartha. Press ESC to stop.


In [66]:
import cv2
import os
from fer import FER

# Initialize the emotion detector
emotion_detector = FER()

def recognize_faces(person_name):
    model_file_path = f'face_trainer/{person_name}.yml'
    
    if not os.path.exists(model_file_path):
        print(f"Model for {person_name} not found. Please train the model first.")
        return

    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read(model_file_path)

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    cap = cv2.VideoCapture(0)

    print(f"Recognizing {person_name}. Press ESC to stop.")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

        for (x, y, w, h) in faces:
            face_region = gray[y:y+h, x:x+w]
            face_id, confidence = recognizer.predict(face_region)

            # Emotion detection
            rgb_face = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            emotion = emotion_detector.detect_emotions(rgb_face)
            
            if len(emotion) > 0:
                dominant_emotion = max(emotion[0]['emotions'], key=emotion[0]['emotions'].get)
                emotion_text = f"Emotion: {dominant_emotion}"
            else:
                emotion_text = "Emotion: Unknown"

            if confidence < 60:
                cv2.putText(frame, f"Matched (Confidence: {round(100 - confidence)}%)", (x+5, y-25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            else:
                cv2.putText(frame, "Not Matched", (x+5, y-25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            cv2.putText(frame, emotion_text, (x+5, y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        cv2.imshow("Face Recognition", frame)

        if cv2.waitKey(1) & 0xFF == 27:  # ESC to break
            break

    cap.release()
    cv2.destroyAllWindows()

# Main function to recognize faces
person_name = input("Enter the name of the person to recognize: ")
recognize_faces(person_name)


Recognizing Siddhartha. Press ESC to stop.


In [67]:
def get_emotion_choice():
    emotions = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
    print("Choose an emotion from the following list:")
    for i, emotion in enumerate(emotions):
        print(f"{i}: {emotion}")
    choice = int(input("Enter the number corresponding to your chosen emotion: "))
    if 0 <= choice < len(emotions):
        return emotions[choice]
    else:
        print("Invalid choice. Defaulting to 'neutral'.")
        return 'neutral'


In [69]:
import cv2
import os
from fer import FER

# Initialize the emotion detector
emotion_detector = FER()

def recognize_faces(person_name, chosen_emotion):
    model_file_path = f'face_trainer/{person_name}.yml'
    
    if not os.path.exists(model_file_path):
        print(f"Model for {person_name} not found. Please train the model first.")
        return

    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read(model_file_path)

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    cap = cv2.VideoCapture(0)

    print(f"Recognizing {person_name} with chosen emotion '{chosen_emotion}'. Press ESC to stop.")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

        for (x, y, w, h) in faces:
            face_region = gray[y:y+h, x:x+w]
            face_id, confidence = recognizer.predict(face_region)

            # Emotion detection
            rgb_face = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            emotion = emotion_detector.detect_emotions(rgb_face)
            
            if len(emotion) > 0:
                detected_emotion = max(emotion[0]['emotions'], key=emotion[0]['emotions'].get)
                emotion_text = f"Emotion: {detected_emotion}"
            else:
                detected_emotion = 'unknown'
                emotion_text = "Emotion: Unknown"

            # Check if the detected emotion matches the chosen emotion
            if confidence < 60 and detected_emotion == chosen_emotion:
                cv2.putText(frame, f"Matched (Confidence: {round(100 - confidence)}%)", (x+5, y-25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            else:
                cv2.putText(frame, "Not Matched", (x+5, y-25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            cv2.putText(frame, emotion_text, (x+5, y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        cv2.imshow("Face Recognition", frame)

        if cv2.waitKey(1) & 0xFF == 27:  # ESC to break
            break

    cap.release()
    cv2.destroyAllWindows()

# Main function to recognize faces with emotion choice
chosen_emotion = get_emotion_choice()
person_name = input("Enter the name of the person to recognize: ")
recognize_faces(person_name, chosen_emotion)


Choose an emotion from the following list:
0: angry
1: disgust
2: fear
3: happy
4: sad
5: surprise
6: neutral
Recognizing Siddhartha with chosen emotion 'happy'. Press ESC to stop.
